In [1]:
from datetime import datetime, timedelta
import numpy as np

In [2]:
def CC_DailyFlow_Historical_Rev(BT_AnnFlow, BT_MonthlyFlow, Precip, AnalogYr, ACOND, dT, dP, dCV):
    
    # Get the length for use below
    HydroLength = len(BT_MonthlyFlow)

    # Future years
    WY = np.loadtxt('./Input/WY.txt')
    
    # Historical BT daily flows
    BTDFlows_load = np.loadtxt('./Input/BT_Historical_DFlows_Catalog.txt', delimiter = ',')
    BTDFlows = np.round(BTDFlows_load, 2)

    # Historical SC Climate
    SC_Climate_load = np.loadtxt('./Input/SC_HistClimateCatalog.txt', delimiter = ',')
    SC_Climate = np.zeros((len(SC_Climate_load),4))
    SC_Climate[:,:3] = SC_Climate_load

    # Go through and sum the historical precip by water year
    for j in range(len(SC_Climate)):
        if SC_Climate[j, 0] == 9:
            WaterYear = SC_Climate[j, 1]
            idx = np.where(SC_Climate[:, 1] == WaterYear)[0][0]
            idx1 = np.where(SC_Climate[:, 1] == WaterYear)[0][-1]
            SC_Climate[idx:idx1+1, 3] = np.sum(SC_Climate[idx:idx1+1, 2])

    # Now do the same for the future climate precip time series
    Precip = np.c_[Precip, np.zeros((len(Precip),2))] # Add column of zeros to fill
    for j in range(HydroLength):
        if Precip[j, 1] == 9:
            if j < 11:
                # Fill a water year array in the Precip variable
                Precip[0:j+1, 4] = Precip[j, 0]
            else:
                # Fill a water year array in the Precip variable
                Precip[j-11:j+1, 4] = Precip[j, 0]
            WaterYear = Precip[j, 4]
            idx = np.where(Precip[:, 4] == WaterYear)[0][0]
            idx1 = np.where(Precip[:, 4] == WaterYear)[0][-1]
            Precip[idx:idx1+1, 5] = np.sum(Precip[idx:idx1+1, 2])

    # Convert monthly rainfall to inches per month
    # This no longer is used in the historical calculation
    # Precip[:, 3] = Precip[:, 2] / 10 / 2.54
    # HC_Hgraph = np.ceil(HydroCond_Ann)
    HC_Hgraph = np.ones((len(BT_AnnFlow), 5))
    HC_Hgraph[:, 0] = BT_AnnFlow[:, 0]

    for j in range(HydroLength):
        if Precip[j, 1] == 9:
            # Replace third column with precip total
            HC_Hgraph[j//12, 2] = Precip[j, 5]
            # This is the flag for the analog historical daily flow year to use
            HC_Hgraph[j//12, 3] = AnalogYr[j]
    
    for j in range(len(HC_Hgraph)):
        WaterYear = HC_Hgraph[j, 3]
        idx = np.where(SC_Climate[:, 1] == WaterYear)[0][-1]
        HC_Hgraph[j, 4] = SC_Climate[idx, 3]

    dateLength = int(np.sum(BT_MonthlyFlow[:, 0]))
    CC_ProjTimeSeries = np.zeros((dateLength, 8))

    # Date at start of time series
    A = np.datetime64(str(int(BT_MonthlyFlow[0, 2])) + '-' + str(int(BT_MonthlyFlow[0, 1])) + '-01')
    B = np.datetime64(str(int(BT_MonthlyFlow[-1, 2])) + '-0' + str(int(BT_MonthlyFlow[-1, 1])) + '-30')
    if ACOND == 1: ##### SAM: I think the else statement could be used in all cases
        ymd = pd.date_range(start = '1937-10-01', end = '2015-09-30')
    else:
        ymd = pd.date_range(start = A, end = B)
    y = np.array([val.year for val in ymd])
    m = np.array([val.month for val in ymd])
    d = np.array([val.day for val in ymd])
    CC_ProjTimeSeries[:, 0] = m #ymd.astype('datetime64[M]').astype(int) % 12 + 1
    CC_ProjTimeSeries[:, 1] = d #ymd.astype('datetime64[D]').astype(int) % 30 + 1
    CC_ProjTimeSeries[:, 2] = y #ymd.astype('datetime64[Y]').astype(int) + 1970

    for j in range(dateLength):
        if CC_ProjTimeSeries[j, 0] >= 10:
            CC_ProjTimeSeries[j, 3] = CC_ProjTimeSeries[j, 2] + 1
        else:
            CC_ProjTimeSeries[j, 3] = CC_ProjTimeSeries[j, 2]

        b1 = str(int(CC_ProjTimeSeries[j, 2]))
        b2 = str(int(CC_ProjTimeSeries[j, 0]))
        str_var = b1 + b2
        CC_ProjTimeSeries[j, 4] = int(str_var)

    lambda_var = 250

    # Let's calculate the daily hydrographs
    for j in range(len(HC_Hgraph)):
        
        Ann_Precip = HC_Hgraph[j, 2]
        WaterYear = HC_Hgraph[j, 0]
        # Analog year
        AYear = HC_Hgraph[j, 3]
        idx = np.where(CC_ProjTimeSeries[:, 3] == WaterYear)[0][0]
        idx1 = np.where(CC_ProjTimeSeries[:, 3] == WaterYear)[0][-1]
        diff1 = idx1 - idx
        
        idx2 = np.where(BTDFlows[:, 3] == AYear)[0][0]
        idx3 = np.where(BTDFlows[:, 3] == AYear)[0][-1]
        diff2 = idx3 - idx2

        if Ann_Precip == 0:
            
            for jj in range(idx, idx1+1):

                # NOTE THIS ASSUMES THAT ALL RECORDS HAVE AT LEAST ONE WATER YEAR WITH PRECIPITATION
                init_value = CC_ProjTimeSeries[idx-1, 5]
                # Use the counter step down the exp function
                decay_rate = np.exp(-(1/lambda_var) * ((jj + 1) - idx))
                CC_ProjTimeSeries[jj, 5] = init_value * decay_rate
                
                if CC_ProjTimeSeries[jj, 5] < 0:
                    CC_ProjTimeSeries[jj, 5] = 0
                    
        else:

            # Just step through and project year by year since everything is in sequential order
            
            if diff2 == 365 and diff1 == 364:

                # Dealing with leap year mismatch here; Feb 29 is 151 days after
                # start of the water year; Basically skip Feb 29th here
                # There is a more elegant way to do this -- find Feb 28th and
                # Mar 1st in any given year
                
                CC_ProjTimeSeries[idx:idx+150+1, 5] = BTDFlows[idx2:idx2+150+1, 4] * (HC_Hgraph[j, 2] / HC_Hgraph[j, 4]) #151
                CC_ProjTimeSeries[idx+151:idx1+1, 5] = BTDFlows[idx2+152:idx3+1, 4] * (HC_Hgraph[j, 2] / HC_Hgraph[j, 4]) #152, 153
            
            elif diff2 == 364 and diff1 == 365:

                # Dealing with leap year mismatch here; Feb 29 is 151 days after
                # start of the water year; Add Feb 29th here
                # There is a more elegant way to do this -- find Feb 28th and
                # Mar 1st in any given year
                
                CC_ProjTimeSeries[idx:idx+151+1, 5] = BTDFlows[idx2:idx2+151+1, 4] * (HC_Hgraph[j, 2] / HC_Hgraph[j, 4])
                CC_ProjTimeSeries[151, 5] = CC_ProjTimeSeries[150, 5]
                CC_ProjTimeSeries[idx+152:idx1+1, 5] = BTDFlows[idx2+151:idx3+1, 4] * (HC_Hgraph[j, 2] / HC_Hgraph[j, 4])
            
            else:

                CC_ProjTimeSeries[idx:idx1+1, 5] = BTDFlows[idx2:idx3+1, 4] * (HC_Hgraph[j, 2] / HC_Hgraph[j, 4])

    m, _ = CC_ProjTimeSeries.shape

    # Correct the flow record at the end of each water year to provide a 
    # smooth transition for year to year. Otherwise abrupt changes can occur.
    # Define a backward looking vector length to spread abrupt flow changes
    # This is for the first case < 1.15; i.e. decrease in flow at WY
    back_1 = 30
    # This is for the second case > 1.15; i.e. increase in flow at WY
    # back_2 = 122
    forward = 122
    forward_2 = 91
    efold_back = 1/15
    efold_forward = 1/500
    peak = 25

    for j in range(m):
        
        if j < m-1:
            
            if CC_ProjTimeSeries[j, 0] == 9 and CC_ProjTimeSeries[j, 1] == 30: # if Sept 30
                
                if CC_ProjTimeSeries[j, 5] / CC_ProjTimeSeries[j+1, 5] > 1.150:
                    
                    anchor_back = CC_ProjTimeSeries[(j+1)-back_1, 5]
                    delta1 = CC_ProjTimeSeries[(j+1)-back_1, 5] - CC_ProjTimeSeries[(j+1), 5]
                    array_back = np.arange(back_1, 0, -1)
                    
                    
                    for jj in np.arange(back_1, 0, -1):

                        CC_ProjTimeSeries[(j+1)-jj, 6] = anchor_back - (delta1 - (delta1 * np.exp(-efold_back*array_back[jj-1])))
                
                if CC_ProjTimeSeries[j+1, 5] / CC_ProjTimeSeries[j, 5] > 1.150:
                    
                    idx = np.where(CC_ProjTimeSeries[j:(j+forward+1), 5] > peak)[0][0]
                    
                    if np.size(idx) > 0:
                        
                        forward_find = j + (idx-2)
                        
                        for jj in range(j, forward_find+2): #used to be +1
                            
                            if jj == j + 1:
                                
                                CC_ProjTimeSeries[jj, 6] = CC_ProjTimeSeries[jj-1, 5] - (efold_forward * CC_ProjTimeSeries[jj-1, 5])
                            
                            else:
                                
                                CC_ProjTimeSeries[jj, 6] = CC_ProjTimeSeries[jj-1, 6] - (efold_forward * CC_ProjTimeSeries[jj-1, 6])
                    
                    elif np.size(idx) == 0:

                        print('Error. No peak found.')
                        
                        for jj in range(j, forward_2+1):
                            
                            if jj == j + 1:
                                
                                CC_ProjTimeSeries[jj, 6] = CC_ProjTimeSeries_NC[jj-1, 5] - (efold_forward * CC_ProjTimeSeries_NC[jj-1, 5])
                            
                            else:
                                
                                CC_ProjTimeSeries[jj, 6] = CC_ProjTimeSeries_NC[jj-1, 6] - (efold_forward * CC_ProjTimeSeries_NC[jj-1, 6])

            else:

                CC_ProjTimeSeries[j,7] = 0

    for j in range(m):
                
        if CC_ProjTimeSeries[j, 6] == 0:
            
            CC_ProjTimeSeries[j, 6] = CC_ProjTimeSeries[j, 5]
    
        if CC_ProjTimeSeries[j, 0] == 2 and CC_ProjTimeSeries[j, 1] == 29: #if Feb 29
            
            CC_ProjTimeSeries[j, 6] = (CC_ProjTimeSeries[j-1, 5] + CC_ProjTimeSeries[j+1, 5]) / 2

    # Go through and sum flow for each month of the record to recalculate the hydrologic condition
    
    HC_Mon = np.ones((len(AnalogYr), 8))
    HC_Mon[:, 0:4] = BT_MonthlyFlow[:, 0:4]
    idx = np.where(CC_ProjTimeSeries[:, 1] == 1)[0]

    for j in range(len(AnalogYr)):
        
        if j < len(AnalogYr)-1:
            
            HC_Mon[j, 4] = np.sum(CC_ProjTimeSeries[idx[j]:idx[j+1]+1, 6])
        
        else:
            
            HC_Mon[j, 4] = np.sum(CC_ProjTimeSeries[idx[j]:, 6])

        # Sum the flows over each water year
        if HC_Mon[j, 1] == 10:
            
            HC_Mon[j, 5] = HC_Mon[j, 4]
        
        else:
            
            HC_Mon[j, 5] = HC_Mon[j, 4] + HC_Mon[j-1, 5]

    # Bring in hydrological conditions limits data
    # fopen('./Input/HydroCondition_Limits.txt', delimiter = '\t')
    HCL = np.loadtxt('./Input/HydroCondition_Limits.txt', delimiter = '\t')

    # Bring in the historical hydrologic condition
    # fopen('./Input/BT_Historical_HC_Catalog.txt')
    Hist_HC = np.loadtxt('./Input/BT_Historical_HC_Catalog.txt', delimiter = ',')

    # It all works because of the counter
    counter = 11
    AnalogYr = np.c_[AnalogYr, np.zeros(len(AnalogYr))] # Add empty column to fill below
    if dT == 0 and dP == 100 and dCV == 1:

        # First fill in the AnalogYr array with a new column of the WY
        for j in range(len(AnalogYr)):
            
            if HC_Mon[j, 1] > 9:
                
                AnalogYr[j, 1] = AnalogYr[j, 0] + 1
                
            else:
                
                AnalogYr[j, 1] = AnalogYr[j, 0]
                
        for j in range(len(HC_Hgraph)):
            
            AYear = HC_Hgraph[j, 3]
            # Use indices to extract the correct HC conditions
            idx = np.where(Hist_HC[:, 2] == AYear)[0][0]
            idx1 = np.where(Hist_HC[:, 2] == AYear)[0][-1]
            HC_Mon[counter-11:counter+1, 6] = Hist_HC[idx:idx1+1, 3]
            counter += 12
            
    else:

        # Calculate the monthly hydrologic condition
        for j in range(len(AnalogYr)):

            # This is an indexing variable
            month = int(HC_Mon[j, 1]) ##### SAM: I think this should have -1 because it is used for indexing

            # Run through the other percentiles to see if the month fits a higher percentile
            for kk in range(5, 1, -1):
                
                if HC_Mon[j, 5] >= HCL[month-1, kk-1]:
                    
                    HC_Mon[j, 6] = kk
                    
                    break
    # Now define the hydrologic condition to implement the flow rules
    # This set of conditions is backward looking at the monthly step
    # So go ahead and shift things ahead one month
    # This will go in the HydroCondition column 6

    # Clean-up some circshift issues from the last operation
    HC_Mon[:, 7] = np.roll(HC_Mon[:, 6], 1)
    HC_Mon[0, 7] = np.nan
    HC_Mon[0, 7] = HC_Mon[0, 6]
    # Assign to the passing variable
    HydroCond_Mon = HC_Mon

    if ACOND == 1:

        # Bring in Climate Change Daily Data Range to build arrays
        # fopen('./Input/Hist_Daily_DateArr_2.txt')
        CCDaily = np.loadtxt('./Input/Hist_Daily_DateArr_2.txt', delimiter = '\t')
        
    else:

        # Bring in Climate Change Daily Date Range to build arrays
        # fopen('./Input/CC_Daily_DateArr_Catalog_Rev.txt')
        CCDaily = np.loadtxt('./Input/CC_Daily_DateArr_Catalog_Rev.txt', delimiter = ',')

    HydroCond_Daily = CCDaily
    HydroCond_Daily_FlowRules = CCDaily

    DailyLength = len(HydroCond_Daily)

    for j in range(HydroLength):
        
        for jj in range(DailyLength):
            
            if HydroCond_Daily[jj, 0] == HydroCond_Mon[j, 1] and HydroCond_Daily[jj, 3] == HydroCond_Mon[j, 3]:
                
                HydroCond_Daily[jj, 4] = HydroCond_Mon[j, 6]
                HydroCond_Daily_FlowRules[jj, 4] = HydroCond_Mon[j, 7]

    # Now do the annual part
    # Bring in Hydrologic Condition Limits Data
    # fopen('./Input/CC_Daily_DateArr_Catalog_Rev.txt')
    AHCL = np.loadtxt('./Input/AnnHydroCondition_Limits.txt', delimiter = '\t')

    ANNHydroLength = len(BT_AnnFlow)
    
    HydroCond_Ann = np.ones((ANNHydroLength, 4))
    HydroCond_Ann[:, 0] = BT_AnnFlow[:, 0]
    HydroCond_Ann[:, 2] = BT_AnnFlow[:, 1]

    # Calculate the monthly hydrologic condition
    for j in range(ANNHydroLength): # Need to start in the second month of the data set

        # Run through the other percentiles to see if the month fits a higher percentile
        for kk in range(5, 1, -1):
            
            if BT_AnnFlow[j, 1] >= AHCL[kk-1]:
                
                HydroCond_Ann[j, 1] = kk
                
                break

    #####
    
    dateLength = int(np.sum(BT_MonthlyFlow[:, 0]))
    SLRMOD7 = np.zeros((dateLength,1))
    SLRMOD7Flows = np.zeros((dateLength,1))
    SLRMOD8 = np.zeros((dateLength,1))
    
    #####
    
    FinalFlow = np.zeros((m, 4))
    FinalFlow[:, 0] = CC_ProjTimeSeries[:, 0]
    FinalFlow[:, 1] = CC_ProjTimeSeries[:, 1]
    FinalFlow[:, 2] = CC_ProjTimeSeries[:, 2]
    FinalFlow[:, 3] = np.round(CC_ProjTimeSeries[:, 6], 1)

    BT = FinalFlow

    return BT, HC_Hgraph, BTDFlows, WY, SC_Climate, HydroCond_Daily, HydroCond_Daily_FlowRules, HydroCond_Ann, HydroCond_Mon, SLRMOD7, SLRMOD8, SLRMOD7Flows